In [1]:
import pandas as pd
import numpy as np

In [2]:
entity_details_table = pd.read_csv('../../DATA/TRANSFORMED/entity_details/financial_statements_register_entity_details.csv')
issues_indicators_table = pd.read_csv('../../DATA/TRANSFORMED/issues_indicators.csv')

C:\Users\matus.rohal\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3433: DtypeWarning: Columns (13,15,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
display(entity_details_table)
display(issues_indicators_table)

,id,annual_reports_list,financial_statements_list,entity_ico,entity_name,establishment_date,legal_form_code,org_size_code,entity_county,data_source,ownership_category_code,entity_region,sknace_code,consolidated,cancellation_date,entity_state,legal_form_RO,establishment_date_RO,cancellation_date_RO,entity_id
0,4.0,"[2217221, 2203737, 2610345, 2570472, 3015276, ...","[276535, 561042, 985585, 1708116, 1974943, 221...",603783.0,"NESS Slovensko, a.s.",1991-01-17,121.0,11.0,SK0102,SUSR,7.0,SK010,62090.0,True,NaN,NaN,121 - Akciová spoločnosť,1991-01-17,NaN,NaN
1,63.0,NaN,NaN,685496.0,"EKOCHEM, a.s. v likvidácií",1990-11-06,121.0,0.0,SK0103,SUSR,2.0,SK010,46150.0,False,2021-11-23,NaN,121 - Akciová spoločnosť,1990-11-06,2021-11-23,NaN
2,74.0,"[1978963, 2470746, 2888840, 3276598, 3660033, ...","[283197, 567704, 992229, 1713093, 1872215, 233...",590797.0,"ZTS Sabinov, a. s.",1990-05-01,121.0,22.0,SK0418,SUSR,2.0,SK041,28150.0,False,NaN,NaN,121 - Akciová spoločnosť,1990-05-01,NaN,NaN
3,184.0,"[2213168, 2618141, 2983268]","[278539, 563022, 987582, 1709653, 1877774, 235...",2313.0,HPK engineering a.s.,1992-05-01,121.0,5.0,SK0422,SUSR,2.0,SK042,71129.0,False,NaN,NaN,121 - Akciová spoločnosť,1992-05-01,NaN,NaN
4,185.0,"[2094769, 2094752, 2424021, 2420972, 2819810, ...","[282224, 566723, 991244, 1712387, 1957495, 232...",677957.0,"CHEMINVEST, a.s.",1992-04-01,121.0,3.0,SK0412,SUSR,2.0,SK041,68200.0,False,NaN,NaN,121 - Akciová spoločnosť,1992-04-01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410569,2096976.0,NaN,NaN,54893526.0,"MERIN SERVICES, s. r. o.",2022-09-15,112.0,0.0,SK0422,SUSR,7.0,SK042,9900.0,False,NaN,NaN,NaN,NaN,NaN,2096976.0
410570,2096979.0,NaN,NaN,54836255.0,ELDA-BAU s. r. o.,2022-09-08,112.0,0.0,SK041A,SUSR,2.0,SK041,25990.0,False,NaN,NaN,NaN,NaN,NaN,2096979.0
410571,2096986.0,NaN,NaN,54834813.0,Derma Medical s.r.o.,2022-09-01,112.0,0.0,SK0322,SUSR,2.0,SK032,96040.0,False,NaN,NaN,NaN,NaN,NaN,2096986.0
410572,2096989.0,NaN,NaN,54807875.0,Cmetal s. r. o.,2022-09-05,112.0,0.0,SK0215,SUSR,2.0,SK021,43210.0,False,NaN,NaN,NaN,NaN,NaN,2096989.0


,entity_ico,year,DPHZ_vat_registration_cancelled,DPHZ_vat_registration_cancelled_1y_off,DPHZ_vat_registration_cancelled_2y_off,RO_cancelled,RO_cancelled_1y_off,RO_cancelled_2y_off,RUZ_cancelled,RUZ_cancelled_1y_off,RUZ_cancelled_2y_off,RUZ_established,RUZ_established_1y_off,RUZ_established_2y_off,RU_konkurz,RU_konkurz_1y_off,RU_konkurz_2y_off,RU_ostatne_konania,RU_ostatne_konania_1y_off,RU_ostatne_konania_2y_off
0,35,2002.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,35,2003.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,35,2004.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,671,1968.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,671,1969.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1240541,448867799,2017.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1240542,448867799,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1240543,2022206285,2009.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1240544,2022206285,2010.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
